In [3]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
import collections
pd.set_option('display.max_rows', 10)
!date

%load_ext autoreload
%autoreload 2

Sat Aug 17 15:33:54 PDT 2019
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import dp_2020_census.data

In [7]:
%cd ..

/ihme/homes/abie/projects/2019/dp_2020_census


# Generate IPUMS counts with age and sex

In [10]:
def load_transform_orig():
    """Tally count from 1940s IPUMS data, stratified by
    state, county, enumeration district, group quarters type, voting age status, race, and ethnicity

    (adapted from dp_2020_census/data.py)

    Results
    -------
    returns pd.DataFrame with the following columns
    ['state', 'county', 'enum_dist', 'gq', 'age', 'sex', 'race', 'eth', 'count']

    Timing
    ------
    7s for 20_000 rows
    70s for 200_000 rows
    predict 17h for full file (170_897_729 rows)
    """

    counts = collections.Counter()

    fname = '/home/j/Project/Models/us_census/EXT1940USCB.dat'
    colspecs = dp_2020_census.data.ipums_colspecs()

    chunksize = 20_000
    iter = pd.read_fwf(fname, list(colspecs), header=None, names=colspecs.index,
                       chunksize=chunksize)
    for i, df_i in enumerate(iter):
        for j, se in df_i.iterrows():
            if se.RECTYPE == 'H': # household record, so update hh info
                state = se.STATEFIP
                county = se.COUNTY
                enum = se.ENUMDIST
                gq = se.GQ
            else: # person record, assume it refers to a person in most recently read household record
                age = se.AGE
                sex = se.SEX
                race = se.RACE
                eth = se.HISPAN

                counts[(state, county, enum, gq, age, sex, race, eth)] += 1

            if (i*chunksize + j) % 10_000 == 0:
                print('.', end=' ', flush=True)

#         if i >= 0: break

    counts = pd.Series(counts).reset_index()
    counts.columns = ['state', 'county', 'enum_dist', 'gq', 'age', 'sex', 'race', 'eth', 'count'] # brittle pattern, don't mess it up

    print()

    return counts

# df = load_transform_orig()

In [11]:
# df

In [ ]:
fname = '/snfs1/Project/Models/us_census/orig_counts_full.csv'
print(fname)
counts = load_transform_orig()
counts.to_csv(fname)


/snfs1/Project/Models/us_census/orig_counts_full.csv


. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 